In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns 
import warnings
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('database.csv')

In [3]:
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,Magnitude Error,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,NaN,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [4]:
df.shape

(23412, 21)

In [5]:
## main features that are required to predict earthquake

df = df[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]

In [6]:
df.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,01/02/1965,13:44:18,19.246,145.616,131.6,6.0
1,01/04/1965,11:29:49,1.863,127.352,80.0,5.8
2,01/05/1965,18:05:58,-20.579,-173.972,20.0,6.2
3,01/08/1965,18:49:43,-59.076,-23.557,15.0,5.8
4,01/09/1965,13:32:50,11.938,126.427,15.0,5.8


In [7]:
df.isnull().sum()

Date         0
Time         0
Latitude     0
Longitude    0
Depth        0
Magnitude    0
dtype: int64

In [8]:
# let's check the type of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       23412 non-null  object 
 1   Time       23412 non-null  object 
 2   Latitude   23412 non-null  float64
 3   Longitude  23412 non-null  float64
 4   Depth      23412 non-null  float64
 5   Magnitude  23412 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.1+ MB


In [9]:
# the date is in the form of string. so let us convert it to datetime

df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%m/%d/%Y')

In [10]:
# let's extract data from date column

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [11]:
## Extracting minutes, seconds, hours from column Time

minutes = []
hours = []
seconds = []

for t in df.Time:
    t  = t.split(":")
    minutes.append(t[1])
    hours.append(t[0])
    seconds.append(t[2])

In [12]:
## creating new columns minutes, hours, seconds

df['minutes'] = minutes
df['hours'] = hours
df['seconds'] = seconds

df['minutes'] = pd.to_numeric(df['minutes'], errors='coerce')
df['hours'] = pd.to_numeric(df['hours'], errors='coerce')
df['seconds'] = pd.to_numeric(df['seconds'], errors='coerce') 

In [13]:
## dropping date and time column from dataset

df.drop(['Date', 'Time'], axis=1, inplace=True)

In [14]:
df.head()

,Latitude,Longitude,Depth,Magnitude,year,month,day,minutes,hours,seconds
0,19.246,145.616,131.6,6.0,1965.0,1.0,2.0,44,13.0,18.0
1,1.863,127.352,80.0,5.8,1965.0,1.0,4.0,29,11.0,49.0
2,-20.579,-173.972,20.0,6.2,1965.0,1.0,5.0,5,18.0,58.0
3,-59.076,-23.557,15.0,5.8,1965.0,1.0,8.0,49,18.0,43.0
4,11.938,126.427,15.0,5.8,1965.0,1.0,9.0,32,13.0,50.0


In [15]:
x = df.drop(['Depth', 'Magnitude'], axis=1)
y = df[['Depth', 'Magnitude']]

In [16]:
X = x.values
y = y.values

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [18]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [19]:
## creating a neural Network model 

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(8,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [20]:
model.compile(optimizer='adam', loss='squared_hinge', metrics=['accuracy'])

In [21]:
## training the model

history = model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data=(x_test, y_test))

Epoch 1/20
257/257 [==============================] - 6s 10ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 2/20
257/257 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 3/20
257/257 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 4/20
257/257 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 5/20
257/257 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 6/20
257/257 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 7/20
257/257 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.9811 - val_loss: nan - val_accuracy: 0.9828
Epoch 8/20
257/257 [===========================